# Evrişimli Sinir Ağları

Önceki ünitede, sınıf tanımını kullanarak çok katmanlı bir sinir ağının nasıl tanımlanacağını öğrenmiştik, ancak bu ağlar geneldi ve bilgisayarla görme görevleri için özelleşmiş değildi. Bu ünitede özellikle bilgisayarla görme için tasarlanmış **Evrişimli Sinir Ağları (CNN'ler)** hakkında bilgi edineceğiz.

Bilgisayarla görme, genel sınıflandırmadan farklıdır çünkü resimde belirli bir nesneyi bulmaya çalışırken, belirli **örüntüleri** ve bunların kombinasyonlarını arayarak imgeyi tarıyoruz. Örneğin, bir kedi ararken, önce bıyık oluşturabilen yatay çizgilere bakabiliriz ve ardından belirli bir bıyık kombinasyonu bize bunun aslında bir kedi resmi olduğunu söyleyebilir. İmge üzerindeki tam konumları değil, belirli modellerin göreli konumu ve varlığı önemlidir.

Örüntüleri çıkarmak için **evrişimli filtreler** kavramını kullanacağız. Ama önce önceki ünitelerde tanımladığımız tüm bağımlılıkları ve fonksiyonları yükleyelim.

In [ ]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from torchinfo import summary
import numpy as np

from pytorchcv import load_mnist, train, plot_results, plot_convolution, display_dataset
load_mnist(batch_size=128)

## Evrişimli filtreler

Evrişimli filtreler, imgenin her pikseli üzerinde çalışan ve komşu piksellerin ağırlıklı ortalamasını hesaplayan küçük pencerelerdir.



Ağırlık katsayı matrisleriyle tanımlanırlar. MNIST el yazısı rakamlarımız üzerinde iki farklı evrişimli filtre uygulama örneği görelim:

In [ ]:
plot_convolution(torch.tensor([[-1.,0.,1.],[-1.,0.,1.],[-1.,0.,1.]]),'Dikey kenar filtresi')
plot_convolution(torch.tensor([[-1.,-1.,-1.],[0.,0.,0.],[1.,1.,1.]]),'Yatay kenar filtresi')

İlk filtre **dikey kenar filtresi** olarak adlandırılır ve aşağıdaki matris ile tanımlanır:
$$
\left(
    \begin{matrix}
     -1 & 0 & 1 \cr
     -1 & 0 & 1 \cr
     -1 & 0 & 1 \cr
    \end{matrix}
\right)
$$
Bu filtre nispeten tekdüze bir piksel alanı üzerinden geçtiğinde, tüm değerler  0'a toplar. Ancak, imgede dikey bir kenarla karşılaştığında, yüksek uç değer üretilir. Bu nedenle yukarıdaki imgelerde dikey kenarların yüksek ve düşük değerlerle temsil edildiğini, yatay kenarların ise ortalamasının alındığını görebilirsiniz.

Yatay kenar filtresi uyguladığımızda tam tersi bir şey olur - yatay çizgiler büyütülür ve dikey çizgiler ortalanır.

Klasik bilgisayarla görmede, öznitelikler oluşturmak için imgeye birden çok filtre uygulandı ve bunlar daha sonra bir sınıflandırıcı oluşturmak için makine öğrenmesi algoritması tarafından kullanıldı. Bununla birlikte, derin öğrenmede, sınıflandırma problemini çözmek için en iyi evrişimli filtreleri **öğrenen** ağlar oluştururuz.

Bunu yapmak için **evrişimli katmanları** tanıtıyoruz.

## Evrişimli Katmanlar

Evrişimli katmanlar, `nn.Conv2d` yapısı kullanılarak tanımlanır. Aşağıdakileri belirtmemiz gerekiyor:
* `in_channels` - girdi kanallarının sayısı. Bizim durumumuzda gri tonlamalı bir imgeyle uğraşıyoruz, bu nedenle girdi kanallarının sayısı 1'dir.
* `out_channels` - kullanılacak filtre sayısı. 9 farklı filtre kullanacağız, bu da ağa senaryomuz için hangi filtrelerin en iyi şekilde çalıştığını keşfetme fırsatı verecektir.
* `kernel_size` kayan pencerenin boyutudur. Genellikle 3x3 veya 5x5 filtreler kullanılır.

En basit CNN, bir evrişimli katman içerecektir. 28x28 girdi boyutu verildiğinde, dokuz adet 5x5 filtre uyguladıktan sonra 9x24x24'lük bir tensör elde edeceğiz (uzamsal boyut daha küçüktür, çünkü 5 uzunluğundaki bir kayma aralığının 28 piksele sığabileceği yalnızca 24 konum vardır).

Evrişimden sonra, 9x24x24 tensörü 5184 boyutunda tek bir vektöre düzleştiriyoruz ve ardından 10 sınıf üretmek için doğrusal katman ekliyoruz. Ayrıca katmanlar arasında `relu` etkinleştirme fonksiyonunu kullanıyoruz.

In [ ]:
class OneConv(nn.Module):
    def __init__(self):
        super(OneConv, self).__init__()
        self.conv = nn.Conv2d(in_channels=1,out_channels=9,kernel_size=(5,5))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(5184,10)

    def forward(self, x):
        x = nn.functional.relu(self.conv(x))
        x = self.flatten(x)
        x = nn.functional.log_softmax(self.fc(x),dim=1)
        return x

net = OneConv()

summary(net,input_size=(1,1,28,28))

Tam bağlı çok katmanlı ağlardaki yaklaşık 80 bin tane ile karşılaştırıldığında, bu ağın yaklaşık 50 bin tane eğitilebilir parametre içerdiğini görebilirsiniz. Bu, daha küçük veri kümelerinde bile iyi sonuçlar elde etmemizi sağlar çünkü evrişimli ağlar çok daha iyi genelleştirir.

In [ ]:
hist = train(net,train_loader,test_loader,epochs=5)
plot_results(hist)

Gördüğünüz gibi, önceki ünitedeki tam bağlı ağlara kıyasla daha yüksek doğruluk ve çok daha hızlı şekilde elde edebiliyoruz.

Neler olup bittiğini biraz daha anlamaya çalışmak için eğitilmiş evrişimli katmanlarımızın ağırlıklarını da görselleştirebiliriz:

In [ ]:
fig,ax = plt.subplots(1,9)
with torch.no_grad():
    p = next(net.conv.parameters())
    for i,x in enumerate(p):
        ax[i].imshow(x.detach().cpu()[0,...])
        ax[i].axis('off')

Bu filtrelerden bazılarının bazı eğik hareketleri tanıyor gibi göründüğünü, diğerlerinin ise oldukça rastgele göründüğünü görebilirsiniz.

## Çok katmanlı CNN'ler ve ortaklama katmanları

İlk evrişimli katmanlar, yatay veya dikey çizgiler gibi ilkel örüntüleri arar, ancak ilkel şekiller gibi daha yüksek seviyeli örüntüleri aramak için bunların üzerine başka evrişimli katmanlar uygulayabiliriz. Daha sonra daha evrişimli katmanlar, bu şekilleri, sınıflandırmaya çalıştığımız son nesneye kadar resmin bazı kısımlarında birleştirebilir.

Bunu yaparken bir püf noktası da uygulayabiliriz: İmgenin uzamsal boyutunu azaltmak. Kayan 3x3 pencerede yatay bir hareket olduğunu tespit ettikten sonra, bunun tam olarak hangi pikselde meydana geldiği çok önemli değildir. Böylece **ortaklama katmanlarından** birini kullanarak imgenin boyutunu "küçültebiliriz":

* **Ortalama Ortaklama** kayan bir pencere alır (örneğin, 2x2 piksel) ve pencere içindeki değerlerin ortalamasını hesaplar.
* **Azami (Maksimum) Ortaklama**, pencereyi maksimum değerle değiştirir. Maksimum ortaklamanın arkasındaki fikir, kayan pencerede belirli bir örüntünün varlığını tespit etmektir.

Bu nedenle, tipik bir CNN'de, imgenin boyutlarını azaltmak için aralarında ortaklama katmanları olan birkaç evrişimli katman olacaktır. Ayrıca filtre sayısını da artıracağız, çünkü örüntüler daha gelişmiş hale geldikçe aramamız gereken daha olası ilginç kombinasyonlar vardır.

![Ortaklama katmanlarına sahip birkaç evrişimli katmanı gösteren bir imge.](../images/cnn-pyramid.png)

Uzamsal boyutların küçülmesi ve öznitelik/filtre boyutlarının artması nedeniyle bu mimariye **piramit mimarisi** de denir.

In [ ]:
class MultiLayerCNN(nn.Module):
    def __init__(self):
        super(MultiLayerCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.fc = nn.Linear(320,10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 320)
        x = nn.functional.log_softmax(self.fc(x),dim=1)
        return x

net = MultiLayerCNN()
summary(net,input_size=(1,1,28,28))

Bu tanımla ilgili birkaç şeye dikkat edin:

* `Flatten` katmanını kullanmak yerine `forward` fonksiyonu içinde `view` fonksiyonu ile tensörü düzleştiriyoruz. Düzleştirme katmanının eğitilebilir ağırlıkları olmadığından, sınıfımızda ayrı bir katman örneği oluşturmamız şart değildir.
* Modelimizde yalnızca bir ortaklama katmanı örneği kullanıyoruz, çünkü bu aynı zamanda eğitilebilir herhangi bir parametre içermez ve bu tek örnek etkili bir şekilde yeniden kullanılabilir.
* Eğitilebilir parametrelerin sayısı (~8.5 bin), önceki durumlardan önemli ölçüde daha azdır. Bunun nedeni, genel olarak evrişimli katmanların birkaç parametreye sahip olması ve son yoğun katman uygulanmadan önce imgenin boyutsallığının önemli ölçüde azalmasıdır. Az sayıda parametrenin modellerimiz üzerinde olumlu etkisi vardır çünkü bu, daha küçük veri kümesi boyutlarında bile aşırı öğrenmeyi önlemeye yardımcı olur.

In [ ]:
hist = train(net,train_loader,test_loader,epochs=5)

Muhtemelen gözlemlemeniz gereken şey, yalnızca bir katmandan daha yüksek doğruluk elde edebildiğimizdir ve çok daha hızlıdır - yalnızca 1 veya 2 dönem. Bu, gelişmiş ağ mimarisinin, neler olup bittiğini anlamak ve imgelerimizden genel örüntüleri çıkarmak için çok daha az veriye ihtiyaç duyduğu anlamına gelir.

## CIFAR-10 veri kümesinden gerçek imgelerle oynama

El yazısı rakamları tanıma problemimiz bir basit problem gibi görünse de, artık daha ciddi bir şey yapmaya hazırız. Farklı nesnelerin resimlerinden oluşan [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) adlı daha gelişmiş veri kümesini keşfedelim. 10 sınıfa ayrılmış 60 bin 32x32 imge içerir.

In [ ]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=14, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=14, shuffle=False)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
display_dataset(trainset,classes=classes)

CIFAR-10 için iyi bilinen bir mimariye [LeNet](https://en.wikipedia.org/wiki/LeNet) adı verilir ve *Yann LeCun* tarafından önerilmiştir. Yukarıda özetlediğimizle aynı ilkeleri takip eder, ana fark 1 yerine 3 renkli girdi kanalıdır.

Ayrıca bu modele bir basitleştirme daha yapıyoruz - çıktı etkinleştirme işlevi olarak `log_softmax` kullanmıyoruz ve sadece son tam bağlı katmanın çıktısını döndürüyoruz. Bu durumda modeli optimize etmek için `CrossEntropyLoss` kayıp işlevini kullanabiliriz.

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16,120,5)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(120,64)
        self.fc2 = nn.Linear(64,10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = nn.functional.relu(self.conv3(x))
        x = self.flat(x)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = LeNet()

summary(net,input_size=(1,3,32,32))

Bu ağın düzgün bir şekilde eğitilmesi önemli miktarda zaman alacaktır ve tercihen GPU etkin hesaplama ile yapılmalıdır.

In [ ]:
opt = torch.optim.SGD(net.parameters(),lr=0.001,momentum=0.9)
hist = train(net, trainloader, testloader, epochs=3, optimizer=opt, loss_fn=nn.CrossEntropyLoss())

3 dönemlik eğitimle elde edebildiğimiz doğruluk çok iyi görünmüyor. Ancak, kör tahminin bize yalnızca %10 doğruluk sağlayacağını ve sorunumuzun aslında MNIST rakam sınıflandırmasından önemli ölçüde daha zor olduğunu unutmayın. Bu kadar kısa bir eğitim süresinde %50'nin üzerine çıkmak iyi bir başarı gibi görünüyor.

## Ana Fikirler

Bu ünitede, bilgisayarla görme sinir ağlarının arkasındaki ana kavramı öğrendik - evrişimli ağlar. İmge sınıflandırmasına, nesne algılamaya ve hatta imge üretme ağlarına güç veren gerçek yaşam mimarilerinin tümü, yalnızca daha fazla katman ve bazı ek eğitim püf noktaları ile CNN'lere dayanmaktadır.